In [ ]:
import requests

##################
# NOTES ON USING REQUESTS WITH FLASK 
#  1. - did NOT need to stringify. the following code does not work: 
# jsonStr = str(jsonDataObj)
# instead just send json=jsonDataObj
# 2. - DO need to  set headers to application/json.
# include "headers={'Content-Type': 'application/json'"
# res = requests.get(API_URL, json=jsonDataObj, headers={'Content-Type': 'application/json'})
##################

jsonDataObj = { "query": "what about inflation", "index": "raydalio" }

# strJsonDataObj = str(jsonDataObj)
# print (strJsonDataObj)

headers = {'Content-type': 'application/json', 'Accept': 'text/plain'}

response = requests.post(API_URL, json=jsonDataObj, headers=headers)

# why does the above request return a 500 error? 

# how do i print the full request that is being sent to the API?
print (response)
print(response.request.url)
print(response.request.body)
print(response.request.headers)
print (response.text)


In [ ]:
from flask import Flask, render_template, request
import os
import gunicorn

# import a function froma  local py file
from answer_query_py import answer_query_with_context, choose_embedding_model, construct_formatted_response_string #TMP 

# Flask 
app = Flask(__name__)

# local params

model = choose_embedding_model() 

# functions to handle request
def chatbot_response(query, indexname,contextPromptHeader='Context: '):
    print ('query: from chatbot response: ', query, 'indexname: ', indexname)
    # return ('tempo response --- query: from chatbot response: ', query, '\nindexname: ', indexname)
    response = answer_query_with_context(query, indexname, model) 
    return construct_formatted_response_string(response) 

def sayhi(msg):
    print ('msg: from sayhi: ', msg)
    return str("Echo: you said: " + msg)

# Flask routes
@app.route('/')
def home():
    return 'Hello from Home World!'

@app.route("/echo", methods=["GET","POST"])
def echo():
    content = request.get_json(silent=True)
    print ('content: ', content)
    userText = content['msg']
    # print ('userText: ', userText)
    response = sayhi(userText)
    return str(response)

# @app.route("/get", methods=["GET","POST"])
@app.route("/say")
def sayhiapi():
    response = 'hi from sayhiapi'

    
    return str(response)

@app.route("/get", methods=["GET","POST"])
# @app.route("/get")
def get():
    content = request.get_json(silent=True)
    print ('content: ', content)

    # Extract request values 
    query = content['query']
    indexname = content['index']
    try: 
        contextPromptHeader = content['contextPromptHeader']
    except:
        contextPromptHeader = 'Context: '
    # extract values from the request within this route function, then pass to subsequent ones 
    # query = request.args.get('query')
    # indexname = request.args.get('index')
    print ('query: ', query, 'indexname: ', indexname, 'contextPromptHeader: ', contextPromptHeader)
    # main function
    response = chatbot_response(query,indexname, contextPromptHeader)
    return str(response)

# run the app

if __name__ == '__main__':
    app.run()